In [1]:
import lmdb
import pandas as pd

In [2]:
protein_pocket_lst = '/data/protein/SKData/KDBNet/data/DAVIS/davis_protein2pdb.yaml'

protein_pocket_dict = {}
with open(protein_pocket_lst, 'r') as pr:
    info_lines = [line.strip() for line in pr.readlines()]
    for line in info_lines:
        k, v = line.split(':')
        protein_pocket_dict[k] = v.strip()


davis_data = '/data/protein/SKData/KDBNet/data/DAVIS/davis_data.tsv'

In [4]:
# davis dir
from rdkit import Chem
sdf_path = '/data/protein/SKData/KDBNet/data/structure/davis_mol3d_sdf'

import pickle as pkl
# open protein lmdb


pocket_path = '/data/protein/SKData/KDBNet_pocket_pdb/pocket.lmdb'
# mol_path = '/data/protein/SKData/Uni-Mol/KinaseData/mol_conf'
# open pocket
penv = lmdb.open(
            pocket_path,
            subdir=False,
            readonly=True,
            lock=False,
            readahead=False,
            meminit=False,
            max_readers=256,
        )
txn = penv.begin()
keys = list(txn.cursor().iternext(values=False))




pocket_idx_dict = {}
for k in keys:
    datapoint_pickled = txn.get(k)
    idx = int(k)
    # print(f'{idx}')
    data_item = pkl.loads(datapoint_pickled)
    pock_name = ''
    pock_name = data_item['pocket'].split('_')[0]
    # print(pock_name)
    if len(data_item['pocket_atoms']) == 0:
        print(f"{data_item['pocket']}, pocket empty")
        continue
    pocket_idx_dict[pock_name] = idx
    # break



lmdb_path = 'kdb_data'
env1 = lmdb.open(lmdb_path, subdir=True, readonly=False, lock=False, readahead=False, meminit=False, map_size=1099511627776)
txn1 = env1.begin(write=True)


# open tsv files
write_cnt = 0
with open('/data/protein/SKData/KDBNet/data/DAVIS/davis_data.tsv') as df:
    lines = df.readlines()[1:] # ommit head
    for line in lines:
        drug_id, protein_id, _, y = line.strip().split()
        drug_file = f'{sdf_path}/{drug_id}.sdf'
        suppl = Chem.SDMolSupplier(drug_file)
        if suppl is None:
            raise IOError("Unable to read SDF file")

        # Iterate through molecules
        for mol in suppl:
            if mol is None:
                continue  # Skip invalid molecules
            
            # Extract atom types and coordinates
            atom_types = [atom.GetSymbol() for atom in mol.GetAtoms()]
            atom_positions = mol.GetConformer().GetPositions()
        
        # get pocket
        pdb_name = protein_pocket_dict[protein_id].split('.')[0]
        pk_idx = pocket_idx_dict[pdb_name]
        
        datapoint_pickled = txn.get(str(pk_idx).encode())
        pk_item = pkl.loads(datapoint_pickled)
        
        data_item = {}
        
        data_item['pocket_atoms'] = pk_item['pocket_atoms']
        data_item['pocket_coords'] = pk_item['pocket_coordinates']
        data_item['lig_atoms'] = atom_types
        data_item['lig_coords'] = atom_positions
        data_item['y'] = float(y)
        # print(atom_types)
        # print(coordinates)
        # print(protein_id)
        txn1.put(str(write_cnt).encode('ascii'), pkl.dumps(data_item))
        write_cnt += 1
        if write_cnt % 1000 == 0:
            print(f'{write_cnt} mols done!')
txn1.commit()
env1.close()

3MY0_LDN_P_600, pocket empty
1000 mols done!
2000 mols done!
3000 mols done!
4000 mols done!
5000 mols done!
6000 mols done!
7000 mols done!
8000 mols done!
9000 mols done!
10000 mols done!
11000 mols done!
12000 mols done!
13000 mols done!
14000 mols done!
